In [9]:
# Translated to .py by Meritxell Pacheco (December 2016)
# Adapted to PandasBiogeme by Nicola Ortelli (November 2019)

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.optimization import newtonTrustRegionForBiogeme
from biogeme.expressions import Beta, DefineVariable, bioDraws, MonteCarlo
from biogeme.models import loglogit, logit, log

df = pd.read_csv("swissmetro.dat", sep = '\t')
database = db.Database("swissmetro", df)

globals().update(database.variables)

# Exclude data
exclude = (( PURPOSE != 1 ) * ( PURPOSE != 3 ) + ( CHOICE == 0 )) > 0
database.remove(exclude)

In [10]:
# Parameters to be estimated
ASC_TRAIN = Beta('ASC_TRAIN', 0, None, None, 1)
ASC_SM    = Beta('ASC_SM',    0, None, None, 0)
ASC_CAR   = Beta('ASC_CAR',   0, None, None, 0)

B_TIME = Beta('B_TIME', 0, None, None, 0)
B_COST = Beta('B_COST', 0, None, None, 0)
B_HE   = Beta('B_HE',   0, None, None, 0)

M_RAIL = Beta('M_RAIL', 0, None, None, 1)
S_RAIL = Beta('S_RAIL', 1, None, None, 0)
XI_RAIL = M_RAIL + S_RAIL * bioDraws('XI_RAIL', 'NORMAL')

# Definition of new variables
TRAIN_COST = DefineVariable('TRAIN_COST', TRAIN_CO * ( GA == 0 ), database)
SM_COST    = DefineVariable('SM_COST', SM_CO * ( GA == 0 ), database)

# Utilities
V_TRAIN = ASC_TRAIN + B_TIME * TRAIN_TT + B_COST * TRAIN_COST + B_HE * TRAIN_HE + XI_RAIL
V_SM    = ASC_SM    + B_TIME * SM_TT    + B_COST * SM_COST    + B_HE * SM_HE + XI_RAIL
V_CAR   = ASC_CAR   + B_TIME * CAR_TT   + B_COST * CAR_CO

V = {1: V_TRAIN, 2: V_SM, 3: V_CAR}
av = {1: TRAIN_AV, 2: SM_AV, 3: CAR_AV}

# Choice model estimation
prob = logit(V, av, CHOICE)
logprob = log(MonteCarlo(prob))
biogeme = bio.BIOGEME(database, logprob, numberOfDraws = 100)
biogeme.modelName = "Error Component 01"
results = biogeme.estimate(algorithm = newtonTrustRegionForBiogeme)

# Results
pandasResults = results.getEstimatedParameters()
print(pandasResults)
print(f"Nbr of observations: {database.getNumberOfObservations()}")
print(f"LL(0) = {results.data.initLogLike:.3f}")
print(f"LL(beta) = {results.data.logLike:.3f}")
print(f"rho bar square = {results.data.rhoBarSquare:.3g}")
print(f"Output file: {results.data.htmlFileName}")

            Value   Std err     t-test       p-value  Rob. Std err  \
ASC_CAR  0.189131  0.077271   2.447642  1.437943e-02      0.079767   
ASC_SM   0.450998  0.069679   6.472553  9.636092e-11      0.093242   
B_COST  -0.010847  0.000518 -20.928409  0.000000e+00      0.000682   
B_HE    -0.005354  0.000964  -5.554277  2.787636e-08      0.000983   
B_TIME  -0.012768  0.000569 -22.423683  0.000000e+00      0.001044   
S_RAIL   0.010995  0.125352   0.087715  9.301034e-01      0.050176   

         Rob. t-test  Rob. p-value  
ASC_CAR     2.371040  1.773811e-02  
ASC_SM      4.836860  1.319061e-06  
B_COST    -15.894028  0.000000e+00  
B_HE       -5.445981  5.152070e-08  
B_TIME    -12.225141  0.000000e+00  
S_RAIL      0.219136  8.265443e-01  
Nbr of observations: 6768
LL(0) = -6989.578
LL(beta) = -5315.382
rho bar square = 0.239
Output file: Error Component 01.html
